# 02 — Exploratory Data Analysis## HumanForYou — Employee Attrition Prediction---### ObjectivePerform a **comprehensive EDA** to:1. Understand the distribution of each feature2. Identify correlations and patterns linked to attrition3. Detect potential biases in sensitive variables (gender, age, marital status)4. Generate visual insights to guide feature engineering and model selection> This notebook expects `merged_data.csv` from **01_Data_Validation_Pipeline**.

## Section 1: Setup & Data Loading

In [ ]:
# ==============================================================================
# IMPORTS AND CONFIGURATION
# ==============================================================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from pathlib import Path

# Plotting config
plt.rcParams.update({
    "figure.figsize": (12, 6),
    "axes.titlesize": 14,
    "axes.labelsize": 12,
    "figure.dpi": 100,
    "font.size": 11,
})
sns.set_theme(style="whitegrid", palette="muted")

# --- Path Configuration ---
_cwd = Path.cwd()
if (_cwd / "data" / "raw").exists():
    PROJECT_ROOT = _cwd
elif (_cwd.parent / "data" / "raw").exists():
    PROJECT_ROOT = _cwd.parent
else:
    PROJECT_ROOT = Path(r"c:\Users\yanis\Documents\CESI\A5\AI Project\HumanForYou")

OUTPUT_DIR = str(PROJECT_ROOT / "outputs")

df = pd.read_csv(f"{OUTPUT_DIR}/merged_data.csv")

# Encode target as binary for analysis
df["Attrition_binary"] = (df["Attrition"] == "Yes").astype(int)

print(f"Dataset loaded: {df.shape[0]} employees x {df.shape[1]} features")
print(f"Attrition: {df['Attrition_binary'].sum()} Yes ({df['Attrition_binary'].mean()*100:.1f}%) / {(df['Attrition_binary']==0).sum()} No")

## Section 2: Target Variable Analysis**Key question**: How imbalanced is the attrition class? This will drive our choice of metrics and resampling strategy.

In [ ]:
# ==============================================================================
# TARGET DISTRIBUTION
# ==============================================================================

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Count plot
counts = df["Attrition"].value_counts()
colors = ["#2ecc71", "#e74c3c"]
axes[0].bar(counts.index, counts.values, color=colors, edgecolor="white", linewidth=1.5)
axes[0].set_title("Attrition Distribution (Count)")
axes[0].set_ylabel("Number of employees")
for i, (idx, val) in enumerate(counts.items()):
    axes[0].text(i, val + 30, str(val), ha="center", fontweight="bold")

# Pie chart
axes[1].pie(counts.values, labels=counts.index, autopct="%1.1f%%", colors=colors,
            startangle=90, explode=(0, 0.05), shadow=True)
axes[1].set_title("Attrition Distribution (%)")

plt.suptitle("Class Imbalance — Attrition", fontsize=16, fontweight="bold")
plt.tight_layout()
plt.savefig(f"{OUTPUT_DIR}/eda_target_distribution.png", bbox_inches="tight")
plt.show()

imbalance_ratio = counts.min() / counts.max()
print(f"Imbalance ratio (minority/majority): {imbalance_ratio:.3f}")
print(f"→ {'Moderate' if imbalance_ratio > 0.2 else 'Significant'} imbalance — consider SMOTE or class_weight adjustments")

## Section 3: Numerical Features AnalysisDistributions, box plots by attrition, and statistical tests to identify significant differences.

In [ ]:
# ==============================================================================
# NUMERICAL FEATURE DISTRIBUTIONS BY ATTRITION
# ==============================================================================

num_cols = df.select_dtypes(include="number").columns.tolist()
num_cols = [c for c in num_cols if c not in ["EmployeeID", "Attrition_binary"]]

# Statistical test: Mann-Whitney U for each numerical feature
print("MANN-WHITNEY U TEST — Numerical features vs Attrition")
print("=" * 65)

stat_results = []
for col in num_cols:
    group_yes = df.loc[df["Attrition_binary"] == 1, col].dropna()
    group_no  = df.loc[df["Attrition_binary"] == 0, col].dropna()
    if len(group_yes) > 0 and len(group_no) > 0:
        u_stat, p_val = stats.mannwhitneyu(group_yes, group_no, alternative="two-sided")
        stat_results.append({"feature": col, "U_statistic": u_stat, "p_value": p_val})

df_stats = pd.DataFrame(stat_results).sort_values("p_value")
df_stats["significant"] = df_stats["p_value"] < 0.05

significant_features = df_stats[df_stats["significant"]]["feature"].tolist()
print(f"\n{len(significant_features)} features significantly different between groups (p < 0.05):")
for _, row in df_stats[df_stats["significant"]].iterrows():
    print(f"  → {row['feature']:30s} p = {row['p_value']:.2e}")

In [ ]:
# ==============================================================================
# BOX PLOTS — Top significant numerical features by attrition
# ==============================================================================

top_features = significant_features[:8]  # top 8
n_plots = len(top_features)
n_cols = 4
n_rows = (n_plots + n_cols - 1) // n_cols

fig, axes = plt.subplots(n_rows, n_cols, figsize=(16, 4 * n_rows))
axes = axes.flatten()

for i, col in enumerate(top_features):
    sns.boxplot(data=df, x="Attrition", y=col, ax=axes[i], palette=colors, order=["No", "Yes"])
    axes[i].set_title(col, fontweight="bold")
    axes[i].set_xlabel("")

# Hide unused axes
for j in range(i + 1, len(axes)):
    axes[j].set_visible(False)

plt.suptitle("Numerical Features — Distribution by Attrition", fontsize=16, fontweight="bold", y=1.02)
plt.tight_layout()
plt.savefig(f"{OUTPUT_DIR}/eda_numerical_boxplots.png", bbox_inches="tight")
plt.show()

## Section 4: Categorical Features AnalysisChi-squared tests and proportional bar charts for each categorical variable.

In [ ]:
# ==============================================================================
# CATEGORICAL FEATURES — Chi-squared tests and attrition rates
# ==============================================================================

cat_cols = df.select_dtypes(include="object").columns.tolist()
cat_cols = [c for c in cat_cols if c != "Attrition"]

print("CHI-SQUARED TEST — Categorical features vs Attrition")
print("=" * 65)

chi2_results = []
for col in cat_cols:
    ct = pd.crosstab(df[col], df["Attrition"])
    chi2, p_val, dof, expected = stats.chi2_contingency(ct)
    chi2_results.append({"feature": col, "chi2": chi2, "p_value": p_val, "dof": dof})

df_chi2 = pd.DataFrame(chi2_results).sort_values("p_value")
df_chi2["significant"] = df_chi2["p_value"] < 0.05

for _, row in df_chi2.iterrows():
    sig = "**" if row["significant"] else "  "
    print(f"  {sig} {row['feature']:25s} χ² = {row['chi2']:8.2f}  p = {row['p_value']:.2e}")

In [ ]:
# ==============================================================================
# ATTRITION RATE BY CATEGORICAL FEATURE
# ==============================================================================

sig_cat = df_chi2[df_chi2["significant"]]["feature"].tolist()
n_plots = len(sig_cat)
n_cols = min(3, n_plots)
n_rows = (n_plots + n_cols - 1) // n_cols

fig, axes = plt.subplots(n_rows, n_cols, figsize=(6 * n_cols, 5 * n_rows))
if n_plots == 1:
    axes = [axes]
else:
    axes = axes.flatten()

for i, col in enumerate(sig_cat):
    rates = df.groupby(col)["Attrition_binary"].mean().sort_values(ascending=False)
    axes[i].barh(rates.index, rates.values, color=sns.color_palette("RdYlGn_r", len(rates)))
    axes[i].set_xlabel("Attrition Rate")
    axes[i].set_title(f"Attrition by {col}", fontweight="bold")
    axes[i].axvline(df["Attrition_binary"].mean(), color="gray", linestyle="--", alpha=0.7, label="Overall")
    axes[i].legend()
    # Add value labels
    for j, val in enumerate(rates.values):
        axes[i].text(val + 0.005, j, f"{val:.1%}", va="center")

for j in range(i + 1, len(axes)):
    axes[j].set_visible(False)

plt.suptitle("Attrition Rate by Category", fontsize=16, fontweight="bold", y=1.02)
plt.tight_layout()
plt.savefig(f"{OUTPUT_DIR}/eda_categorical_attrition.png", bbox_inches="tight")
plt.show()

## Section 5: Correlation AnalysisHeatmap of numerical features and point-biserial correlations with the target.

In [ ]:
# ==============================================================================
# CORRELATION HEATMAP
# ==============================================================================

# Select numerical columns (including binary target)
corr_cols = [c for c in num_cols if c not in ["EmployeeID"]] + ["Attrition_binary"]
corr_matrix = df[corr_cols].corr()

# Full heatmap
fig, ax = plt.subplots(figsize=(16, 14))
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
sns.heatmap(corr_matrix, mask=mask, annot=True, fmt=".2f", cmap="RdBu_r",
            center=0, vmin=-1, vmax=1, linewidths=0.5, ax=ax,
            annot_kws={"size": 8})
ax.set_title("Feature Correlation Matrix", fontsize=16, fontweight="bold")
plt.tight_layout()
plt.savefig(f"{OUTPUT_DIR}/eda_correlation_heatmap.png", bbox_inches="tight")
plt.show()

In [ ]:
# ==============================================================================
# TOP CORRELATIONS WITH ATTRITION
# ==============================================================================

target_corr = corr_matrix["Attrition_binary"].drop("Attrition_binary").abs().sort_values(ascending=False)

fig, ax = plt.subplots(figsize=(10, 8))
colors_corr = ["#e74c3c" if corr_matrix.loc[feat, "Attrition_binary"] > 0 else "#3498db" 
               for feat in target_corr.index]
ax.barh(target_corr.index, target_corr.values, color=colors_corr)
ax.set_xlabel("|Correlation| with Attrition")
ax.set_title("Feature Correlation with Attrition (absolute)", fontweight="bold")
ax.invert_yaxis()

# Add a legend
from matplotlib.patches import Patch
ax.legend(handles=[Patch(color="#e74c3c", label="Positive (↑ attrition)"),
                   Patch(color="#3498db", label="Negative (↓ attrition)")],
          loc="lower right")

plt.tight_layout()
plt.savefig(f"{OUTPUT_DIR}/eda_target_correlations.png", bbox_inches="tight")
plt.show()

print("Top 10 features correlated with Attrition:")
for feat, val in target_corr.head(10).items():
    direction = "+" if corr_matrix.loc[feat, "Attrition_binary"] > 0 else "−"
    print(f"  {direction} {feat:30s} |r| = {val:.3f}")

## Section 6: Sensitive Variables — Equity Analysis**Ethical checkpoint**: Examine the distribution of sensitive variables (Gender, Age, MaritalStatus) and their relationship with attrition to identify potential biases.> This analysis feeds directly into the **ethics deliverable** (ALTAI Requirement 5: Diversity, Non-Discrimination and Equity).

In [ ]:
# ==============================================================================
# SENSITIVE VARIABLES — EQUITY ANALYSIS
# ==============================================================================

sensitive_vars = ["Gender", "MaritalStatus", "Age"]

fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# Gender × Attrition
ct_gender = pd.crosstab(df["Gender"], df["Attrition"], normalize="index") * 100
ct_gender.plot(kind="bar", stacked=True, ax=axes[0], color=colors)
axes[0].set_title("Attrition by Gender", fontweight="bold")
axes[0].set_ylabel("Percentage (%)")
axes[0].legend(title="Attrition")
axes[0].tick_params(axis="x", rotation=0)

# MaritalStatus × Attrition
ct_marital = pd.crosstab(df["MaritalStatus"], df["Attrition"], normalize="index") * 100
ct_marital.plot(kind="bar", stacked=True, ax=axes[1], color=colors)
axes[1].set_title("Attrition by Marital Status", fontweight="bold")
axes[1].set_ylabel("Percentage (%)")
axes[1].legend(title="Attrition")
axes[1].tick_params(axis="x", rotation=0)

# Age distribution by Attrition
sns.histplot(data=df, x="Age", hue="Attrition", kde=True, ax=axes[2],
             palette={"No": "#2ecc71", "Yes": "#e74c3c"}, alpha=0.6)
axes[2].set_title("Age Distribution by Attrition", fontweight="bold")

plt.suptitle("Sensitive Variables — Equity Analysis", fontsize=16, fontweight="bold", y=1.02)
plt.tight_layout()
plt.savefig(f"{OUTPUT_DIR}/eda_equity_analysis.png", bbox_inches="tight")
plt.show()

# Disparate impact calculation
print("DISPARATE IMPACT — Gender")
print("=" * 65)
rate_male   = df.loc[df["Gender"] == "Male",   "Attrition_binary"].mean()
rate_female = df.loc[df["Gender"] == "Female", "Attrition_binary"].mean()
di = min(rate_male, rate_female) / max(rate_male, rate_female)
print(f"  Attrition rate Male:   {rate_male:.3f}")
print(f"  Attrition rate Female: {rate_female:.3f}")
print(f"  Disparate Impact ratio: {di:.3f} {'✓ (> 0.8 — 4/5 rule)' if di > 0.8 else '⚠ (< 0.8 — potential bias)'}")

## Section 7: Key EDA Findings — Summary

In [ ]:
# ==============================================================================
# EDA SUMMARY
# ==============================================================================

print("=" * 65)
print("EDA KEY FINDINGS")
print("=" * 65)
print(f"""
1. CLASS IMBALANCE
   Attrition rate: ~{df['Attrition_binary'].mean()*100:.0f}% — moderate imbalance.
   → Recommendation: Use stratified splits, consider SMOTE, optimize for Recall/F1.

2. SIGNIFICANT NUMERICAL FEATURES (p < 0.05)
   {', '.join(significant_features[:5])}...
   → These should be prioritized in feature selection.

3. SIGNIFICANT CATEGORICAL FEATURES (p < 0.05)
   {', '.join(sig_cat)}
   → Encode carefully; consider target encoding for high-cardinality features.

4. CORRELATION HIGHLIGHTS
   Top positive: features pushing toward attrition
   Top negative: features protecting against attrition
   → Check multicollinearity for highly correlated feature pairs.

5. EQUITY OBSERVATIONS
   Gender disparate impact ratio: {di:.3f}
   → {'Acceptable' if di > 0.8 else 'Requires mitigation'} per 4/5 rule.
   → Document in ethics deliverable.
""")
print("\n✓ EDA complete — proceed to 03_Feature_Engineering.ipynb")